In [51]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

In [111]:
import pdfminer
import pdfplumber

In [122]:
import time
import os
import fitz
import PyPDF2
from docx import Document

def count_tokens(text: str) -> int:
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    return len(token_ids)

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_pdf_pypdf2(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

def extract_text_from_pdf_fitz(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text("text") + "\n"
    return text

def extract_text_from_pdf_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def extract_text_from_pdf_pdfminer(pdf_path):
    text = extract_text(pdf_path)
    return text

def process_resume(text, question):
    input_text_template = f"""You are a resume parser. Extract the correct response from the resume for the following question:
                            Based on the following resume, answer the question:
                            
                            Resume:
                            {{resume_text}}
                            
                            Question: {question}
                            Answer:
                            """

    input_text = input_text_template.format(resume_text=text)
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    sample_outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=500,
        top_k=50,
        top_p=0.95,
        temperature=0.3, 
        num_return_sequences=1
    )

    return tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

def extract_details_from_resume(resume_text):
    question = "What are the personal information details from the given resume? (Include Name, Phone Number, Email ID, and Location)" 
    start_time = time.time()
    response = process_resume(resume_text, question)
    end_time = time.time()
    
    return response


resume_files = [
    'test_resumes/Rencata_ACHUDHAN_20240509115030 copy.pdf',  
    
]

overall_start_time = time.time()
for file_path in resume_files:
    if file_path.endswith('.pdf'):
        resume_text = extract_text_from_pdf_pdfminer(file_path)
    elif file_path.endswith('.docx'):
        resume_text = extract_text_from_docx(file_path)
       
    else:
        print(f"Unsupported file format for {file_path}")
        continue
    print(resume_text)
    start_time = time.time()
    details = extract_details_from_resume(resume_text)
    end_time = time.time()
    
    print(f"Resume: {file_path}")
   
    print("Total Tokens: ", count_tokens(resume_text))
    print("Personal Information:", details)  
    print(f"Time taken to generate response: {end_time - start_time:.4f} seconds")
    print("\n" + "=" * 50 + "\n")

overall_end_time = time.time()
print("-----------------")
print(f"Overall Time taken to generate response: {overall_end_time - overall_start_time:.4f} seconds")
print("-----------------")


ACHUDHAN

 Harur, Dharmapuri
 9360593077
 3achudhan@gmail.com

 Skills

Team Management

Problem Solving

Analytical Thinking

Decision making

Excellent Communication

 Interests

Database Management

Web Development

 Languages

English

Tamil

 Objective

Self motivated and enthusiastic graduate seeking an opportunity
to work and to prove my skills and to learn new ideas and
technology the betterment of myself and to the organization.

 Experience

Ashok Leyland

CL

Working as CL in Hosur 

 Education

Annamalai University

Bachelor of Engineering 

7.1

Kongu Polytechnic College

Diploma 

71.33

2022-10-01

- 2023-01-

20

2022

2019

Jayam Matric Higher Secondary School

2017

SSLC 

75

 Projects

Fabrication of Chip Collecting Vehicles

Objective of the work is as the development of Mechanically

driven Magentic chip collector machine and does not rely on

electic power 

 HOBBIES

Painting 

Travelling 

 TECHNICAL SKILLS

HTML5

Css3

JSON

SQL

Javascript 

 IN

SINGLE SHOT PROMPT

In [109]:



def count_tokens(text: str) -> int:
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    return len(token_ids)

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])
def extract_text_from_pdf_pdfminer(pdf_path):
    text = extract_text(pdf_path)
    return text

def process_resume(text, question):
    few_shot_examples = [
        """Resume:    
        A. ABBAS
        Email Id: - mhdabbas2930@gmail.com
        Contact: - +91 9962078071
        SUMMARY:
        2.2 years of experience in IT-Staffing field as US IT Recruiter for the US Market. 
        Answer: {{
            "Name": "ABBAS",
            "Phone Number": "+91 9962078071",
            "Email ID": "mhdabbas2930@gmail.com",
            "Location": "Not-found"
        }}""",

         """Resume:    
        Aarthi Rajendran - Web Developer
        Email: A arthiyuvi@gmail.com Mobile No: +91 -7200749779
        Professional Profile 
        IT Professional with over 7 years of experience in developing, supporting and managing web applications in Airline domain, Health Insurance and Banking sectors
        Good programming experience in Javascript technologies like Angular, Vue JS, Knockout JS 
        Motivated to take challenging tasks and eager to solve complex technical issues
        Always focused to provide best IT solution for my customer
        Desired to work in team with good spirit and pass key functional and technical knowledge to everyone
        Keen to learn new features in application and keep myself update on the latest programming skills
        Followthe agreed process designated by my customer to solve issues and capable to bold decisions whenever I need toof experience in IT-Staffing field as US IT Recruiter for the US Market. 
        Answer: {{
            "Name": "Aarthi Rajendran",
            "Phone Number": "+91 -7200749779",
            "Email ID": "A arthiyuvi@gmail.com",
            "Location": "Not-found"
        }}"""
        
        
       
    ]

    examples_str = "\n\n".join(few_shot_examples)

    input_text_template = f"""You are a resume parser. Use the following examples to extract personal information from the resume:

{examples_str}

Resume:
{{resume_text}}

Question: {question}

Answer in JSON format:
"""

    input_text = input_text_template.format(resume_text=text)
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    sample_outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=500,
        top_k=50,
        top_p=0.95,
        temperature=0.1,
        num_return_sequences=1
    )

    return tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

def extract_details_from_resume(resume_text):
    question = "What are the personal information details from the given resume? (Include Name, Phone Number, Email ID, and Location)"
    
    start_time = time.time()
    response = process_resume(resume_text, question)
    end_time = time.time()
    
    return response


resume_files = [
    'test_resumes/Rencata_ACHUDHAN_20240509115030 copy.pdf',  
  
]

overall_start_time = time.time()
for file_path in resume_files:
    if file_path.endswith('.pdf'):
        resume_text = extract_text_from_pdf_pdfminer(file_path)
    elif file_path.endswith('.docx'):
        resume_text = extract_text_from_docx(file_path)
    else:
        print(f"Unsupported file format for {file_path}")
        continue
    
    start_time = time.time()
    details = extract_details_from_resume(resume_text)
    end_time = time.time()
    
    print(f"Resume: {file_path}")
    print(resume_text)
    print("Total Tokens: ", count_tokens(resume_text))
    print("Personal Information:", details)  
    print(f"Time taken to generate response: {end_time - start_time:.4f} seconds")
    print("\n" + "=" * 50 + "\n")

overall_end_time = time.time()
print("-----------------")
print(f"Overall Time taken to generate response: {overall_end_time - overall_start_time:.4f} seconds")
print("-----------------")


Resume: test_resumes/Rencata_ACHUDHAN_20240509115030 copy.pdf
ACHUDHAN

 Harur, Dharmapuri
 9360593077
 3achudhan@gmail.com

 Skills

Team Management

Problem Solving

Analytical Thinking

Decision making

Excellent Communication

 Interests

Database Management

Web Development

 Languages

English

Tamil

 Objective

Self motivated and enthusiastic graduate seeking an opportunity
to work and to prove my skills and to learn new ideas and
technology the betterment of myself and to the organization.

 Experience

Ashok Leyland

CL

Working as CL in Hosur 

 Education

Annamalai University

Bachelor of Engineering 

7.1

Kongu Polytechnic College

Diploma 

71.33

2022-10-01

- 2023-01-

20

2022

2019

Jayam Matric Higher Secondary School

2017

SSLC 

75

 Projects

Fabrication of Chip Collecting Vehicles

Objective of the work is as the development of Mechanically

driven Magentic chip collector machine and does not rely on

electic power 

 HOBBIES

Painting 

Travelling 
